In [1]:
import os
from dotenv import load_dotenv
from pprint import pprint
load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['NEO4J_URI'] = os.getenv('NEO4J_URI')
os.environ['NEO4J_USERNAME'] = os.getenv('NEO4J_USERNAME')
os.environ['NEO4J_PASSWORD'] = os.getenv('NEO4J_PASSWORD')
os.environ['together_api_key'] = os.getenv('TOGETHER_API_KEY')


In [2]:
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain_openai import OpenAIEmbeddings

vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated procedure. ('db.create.setVectorProperty' has been replaced by 'db.create.setNodeVectorProperty')} {position: line: 1, column: 70, offset: 69} for query: "UNWIND $data AS row MATCH (n:`Document`) WHERE elementId(n) = row.id CALL db.create.setVectorProperty(n, 'embedding', row.embedding) YIELD node RETURN count(*)"


In [3]:
from prompt_templates import extract_entities_prompt,  extract_entities_parser
from llm_model import LLM

In [4]:
gpt_llm = LLM('OpenAI')
llm = gpt_llm.llm
structured_llm = extract_entities_prompt | llm | extract_entities_parser


In [6]:
from langchain_community.graphs import Neo4jGraph
graph = Neo4jGraph()




['Wheezing', 'treatment']
Asthma - HAS_SYMPTOM -> Symptoms
Asthma - HAS_SYMPTOM -> Airflow Obstruction
Asthma - HAS_SYMPTOM -> Bronchospasms
Asthma - HAS_SYMPTOM -> Wheezing
Asthma - HAS_SYMPTOM -> Coughing
Asthma - HAS_SYMPTOM -> Chest Tightness


In [7]:
from advanced_rag import retriever

In [10]:
from langchain_core.runnables import (
    RunnableParallel,
    RunnablePassthrough,
)
from langchain_core.output_parsers import  StrOutputParser


In [26]:
from prompt_templates import rag_prompt

chain = (
    RunnableParallel(
        {
            "context": lambda x: retriever(graph = x['graph'],vector_index=x['vector_index'], structured_llm=x['structured_llm'], question=x['question']),
            "question": RunnablePassthrough(),
        }
    )
    | rag_prompt
    | llm
    | StrOutputParser()
)

chain.invoke({'question':'How many people have asthma', "graph":graph, "vector_index":vector_index, "structured_llm":structured_llm})


Search query: How many people have asthma
['people', 'asthma']


'262 million people worldwide have asthma.'